# Referendum Results

This notebook scrapes the voting results in the membership of the European Union on 23rd June, 2016. This is scraped from the BBC website

In [1]:
from collections import defaultdict
from decimal import Decimal
import string
import requests
from bs4 import BeautifulSoup

In [2]:
# Set a constant for the alphabet's letters
ALPHABET = string.ascii_lowercase

In [3]:
# Set the base url in readiness to append the letter
BASE_URL = 'https://www.bbc.co.uk/news/politics/eu_referendum/results/local/'

In [4]:
# Set initial variables and results dictionary
results = defaultdict(list)
leave_votes, remain_votes = 0, 0

In [5]:
# Scrape content from each lettered page from the BBC website
# Print the status code returned from each page for review
for letter in ALPHABET:
    page_response = requests.get(f"{BASE_URL}{letter}", timeout=5)
    print(f"{'URL:'} {BASE_URL}{letter}{' | Status Code: '}{page_response.status_code}")
    if page_response:
        page_content = BeautifulSoup(page_response.content, "html.parser")
        areas = page_content.find_all('div', attrs={'class': 'eu-ref-result-bar'})
        for area in areas:
            results['area_name'].append(area.find('h3').getText())
            cleaned_leave_votes = int(area.find_all('div',
                                    {'class': 'eu-ref-result-bar__votes'}
                                    )[0].string.strip().split('\n')[0].strip().replace(',', ''))
            results['leave_votes'].append(cleaned_leave_votes)
            cleaned_remain_votes = int(area.find_all('div',
                                    {'class': 'eu-ref-result-bar__votes'}
                                    )[1].string.strip().split('\n')[0].strip().replace(',', ''))
            results['remain_votes'].append(cleaned_remain_votes)
            area_votes = cleaned_leave_votes + cleaned_remain_votes
            results['area_votes'].append(area_votes)
            results['leave_percent'].append(f"{cleaned_leave_votes / area_votes:.1%}")
            results['remain_percent'].append(f"{cleaned_remain_votes / area_votes:.1%}")
            results['turnout'].append(area.find(
                'div', {'class': 'eu-ref-result-bar__turnout'})
                .getText().replace('Turnout: ', '')
            )
            leave_votes = sum(Decimal(num) for num in results['leave_votes'])
            remain_votes = sum(Decimal(num) for num in results['remain_votes'])
    else:
        continue

URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/a | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/b | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/c | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/d | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/e | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/f | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/g | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/h | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/i | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/j | Status Code: 404
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/k | Status Code: 200
URL: https

In [6]:
# Print a totals summary for the number of leave and remain votes
print(f"Leave votes: {leave_votes:,}\nRemain votes: {remain_votes:,}")

Leave votes: 17,410,742
Remain votes: 16,141,241


In [7]:
# Zip together the individual components of an area's result
area_results = zip(
    results['area_name'],
    results['leave_votes'],
    results['leave_percent'],
    results['remain_votes'],
    results['remain_percent'],
    results['turnout'],
)

In [8]:
# Print the results for each area
for area in area_results:
    print(area)